In [ ]:
# Imports
import os;
import sys;
import numpy as np;
from sklearn.linear_model import LinearRegression;

In [4]:
# To read synset vectors generated by Sudha's tool
def read_sudha_format( fname):
    model = dict();
    f = open( fname, 'r');
    lcnt = 0;
    for line in f:
        if lcnt == 0:
            lcnt += 1;
            continue;
        tokens = line.strip().split( ' ');
        key = tokens[ 0];
        val = np.asarray( [ float( token) for token in tokens[ 1:]]);
        word, pos, sid = key.split( '@');
        if pos == 'noun':
            pos = 'n';
        if pos == 'adjective' or pos == 'adj':
            pos = 'a';
        if pos == 'verb':
            pos = 'v';
        if pos == 'adverb' or pos == 'adv':
            pos = 'r';
        mkey = sid + '_' + pos;
        model[ mkey] = val;
    return model;

# To remove almost zero vectors
def filter_zero_vectors( model, eps):
    to_remove = [];
    for key in model:
        val = model[ key];
    if np.linalg.norm( val) < eps:
        to_remove.append( key);
    for elem in to_remove:
        del model[ elem];
    return model;

# create dictionary of pos specific models
def pos_dict( model):
    ret_dict = dict();
    pos_list = [ 'a', 'n', 'r', 'v'];
    for pos in pos_list:
        ret_dict[ pos] = dict();
    for key in model:
        if 'a' in key:
            ret_dict[ 'a'][ key] = model[ key];
        if 'n' in key:
            ret_dict[ 'n'][ key] = model[ key];
        if 'r' in key:
            ret_dict[ 'r'][ key] = model[ key];
        if 'v' in key:
            ret_dict[ 'v'][ key] = model[ key];
    

# Read a specific link file
def read_link_file( lfname):
    f = open( lfname, 'r');
    link_data = [];
    for line in f:
        tokens = line.strip().split('\t');
        link_data.append( ( tokens[ 0], tokens[ 2]));
    f.close();
    return link_data;

# Filter link list data by availability of synset vectors
def filter_link_list( link_list, src_model, tgt_model, pos):
    ret_list = [];
    for tup in link_list:
        src_id = tup[ 0];
        tgt_id = tup[ 1];
        src_str = str( src_id) + '_' + pos;
        tgt_str = str( tgt_id) + '_' + pos;
        if src_str in src_model and tgt_str in tgt_model:
            ret_list.append( tup);
    return ret_list;

# training models for all dimensions of the tgt language synset vector
def train_s2t( train_list, inp_m, out_m, pos = 'n', bias_p = False):
    ivlist = [];
    ovlist = [];
    for tup in train_list:
        k1, k2 = tup;
        ivlist.append( inp_m[ k1]);
        ovlist.append( out_m[ k2]);
    i_data = np.asarray( ivlist, dtype = 'f4');
    o_data = np.asarray( ovlist, dtype = 'f4');
    dim = len( o_data[ 0]);
    #print( dim);
    dmodel_list = [];
    print( 'Training', flush = True);
    for i in range( dim):
        #print( 'Dim = ', i, flush = True);
        X = i_data;
        Y = o_data[ :, i];
        cur_model = LinearRegression( fit_intercept = bias_p);
        cur_model.fit( X, Y);
        dmodel_list.append( cur_model);
    return dmodel_list;

# applying models to test data
def apply_model( model_list, test_list, inp_m, out_m, pos = 'n'):
    ivlist = [];
    ovlist = [];
    for tup in test_list:
        k1, k2 = tup;
        ivlist.append( inp_m[ k1]);
        #ovlist.append( out_m[ k2]);
    i_data = np.array( ivlist, dtype = 'f4');
    dim = len( ovlist[ 0]);
    ypred = [];
    print( 'Translating', flush = True);
    for i in range( dim):
        #print( 'Dim = ', i, flush = True);
        X = i_data;
        cur_model = model_list[ i];
        Yp = cur_model.predict( X);
        ypred.append( Yp);
    pred_data = ypred[ 0];
    for i in range( 1, dim):
        pred_data = np.column_stack( ( pred_data, ypred[ i]));
    #print( np.shape( pred_data));
    return pred_data;

# computing accuracy of a fold
def get_acc( test_list, inp_m, out_m, pred_data, pos = 'n'):
    print( 'Sim Scoring', flush = True);
    ivlist = [];
    ovlist = [];
    ptr = 0;
    k2id = dict();
    id2k = dict();
    for key in out_m:
        ovlist.append( out_m[ key]);
        k2id[ key] = ptr;
        id2k[ ptr] = key;
        ptr += 1;
    o_data = np.asarray( ovlist, dtype = 'f4');
    no_data = ( o_data.T / np.linalg.norm( o_data, axis = 1)).T;
    npred_data = ( pred_data.T / np.linalg.norm( pred_data, axis = 1)).T;
    sim_mat = npred_data.dot( no_data.T);
    r, c = np.shape( sim_mat);
    #trim_sim_mat = np.sort( sim_mat)[ :, ::-1];
    trim_ind_mat = np.argsort( sim_mat)[ :, ::-1];
    indx = [];
    #posl = [];
    print( 'Ranking', flush = True);
    for i in range( r):
        true = test_list[ i][ 1];
        pos = test_list[ i][ 2];
        tlist = trim_ind_mat[ i].tolist();
        tind = tlist.index( k2id[ true]);
        indx.append( tind);
        #posl.append( pos);
    acc_01 = sum( [ elem < 1 for elem in indx]);
    acc_03 = sum( [ elem < 3 for elem in indx]);
    acc_05 = sum( [ elem < 5 for elem in indx]);
    acc_08 = sum( [ elem < 8 for elem in indx]);
    acc_10 = sum( [ elem < 10 for elem in indx]);
    acc_20 = sum( [ elem < 20 for elem in indx]);
    acc_50 = sum( [ elem < 50 for elem in indx]);
    acc_100 = sum( [ elem < 100 for elem in indx]);
    #avg_cnt = sum( [ elem == true for elem in indx])
    print( acc_01 / len( indx));
    print( acc_03 / len( indx));
    print( acc_05 / len( indx));
    print( acc_08 / len( indx));
    print( acc_10 / len( indx));
    print( acc_20 / len( indx));
    print( acc_50 / len( indx));
    print( acc_100 / len( indx));
    print( min( indx), max( indx), ptr);

# cross validation
def cross_validation( inp_m, out_m, link_list, pos = 'n', bias_p = False):
    fold_beg = [ 0] * 10;
    fold_end = [ 0] * 10;
    for i in range( 1, 10):
        fold_end[ i - 1] = (len( link_list) // 10) * i;
        fold_beg[ i] = (len( link_list) // 10) * i;
    fold_end[ 9] = len( link_list);
    for fold_ptr in range( 10):
        print( 'Running fold ', fold_ptr, flush = True);
        train_list = [];
        test_list = [];
        for i in range( 10):
            if i != fold_ptr:
                train_list.extend( link_list[ fold_beg[ i] : fold_end[ i]]);
            else:
                test_list.extend( link_list[ fold_beg[ i] : fold_end[ i]]);
        dmodel_list = train_s2t( train_list, inp_m, out_m, pos, bias_p);
        pred_data = apply_model( dmodel_list, test_list, inp_m, out_m, pos);
        get_acc( test_list, inp_m, out_m, pred_data, pos);